# Measurements in Multi-Qubit Systems

This kata will introduce you to measurements done on multi-qubit systems, and how to implement such measurements in Qiskit.
This will include measuring a single qubit in a multi-qubit system, as well as measuring multiple qubits simultaneously.

**This kata covers the following topics:**

- Measuring a single qubit in a multi-qubit system
- Measuring multiple qubits simultaneously
- Implementing different kinds of measurements in Qiskit

**What you should know to start working on this kata:**

- Basic concepts of linear algebra
- Single and multi-qubit systems
- Basic single and multi-qubit gates
- Single-qubit system measurements

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from test_MultiQubitMeasurements import exercise

## Types of measurements

There are several types of measurements you can perform on an $n$-qubit system ($n>1$):

- Measuring all the qubits simultaneously in an orthogonal basis ($2^n$ possible outcomes). As you'll see below, this is a direct generalization of orthogonal basis measurements done in single-qubit systems introduced in the previous kata.
- Partial measurement: measuring $m$ qubits out of $n$, for $m<n$ ($2^m$ possible outcomes). Partial measurements involve a partial collapse of the system's wave function, since only some qubits are measured.
- Joint measurement: measuring a joint property of all $n$ qubits ($2$ possible outcomes).

The following sections discuss the first two concepts and leave joint measurements for a later kata.

## Full measurements: measurements in multi-qubit bases

Consider a system consisting of $n\geq1$ qubits. The wave function of such a system belongs to a vector space of dimension $2^n$. Thus, the vector space is spanned by an orthogonal basis, such as the computational basis which consists of the vectors $\ket{0\dotsc0}, \dotsc, \ket{1\dotsc 1}$. For generality, consider an arbitrary orthonormal basis, which is denoted by $\{ \ket{b_0}, \ket{b_1}, \dotsc, \ket{b_{2^n-1}} \}$.

Then, the state $\ket{\psi}$ of the multi-qubit system can be expressed as a linear combination of the $2^n$ basis vectors $\ket{b_i}$. That is, there exist complex numbers $c_0,c_1,\dotsc, c_{2^n-1}$ such that

$$
\ket{\psi} = \sum_{i=0}^{2^n-1} c_i\ket{b_i} \equiv \begin{pmatrix} c_0 \\ c_1 \\ \vdots \\ c_{2^n-1} \end{pmatrix}
$$

In line with the usual convention, let's choose the wave function to be normalized, so that $|c_0|^2 + \dotsc + |c_{2^n-1}|^2 =1$. Then, a quantum measurement in the basis $\{ \ket{b_0}, \ket{b_1}, \dotsc, \ket{b_{2^n-1}} \}$ satisfies the following rules:

- The measurement outcome $b_i$ occurs with probability $|c_i|^2$.
- Whenever the measurement outcome is $b_i$, the wave function collapses to the state $\ket{b_i}$. That is, the post-measurement state of the system is equal to $\ket{b_i}$.

This can be summarized in the following table:

| Measurement outcome | Probability of outcome | State after measurement |
| - | - | - |
| $b_i$ | $\|c_i\|^2$ | $\ket{b_i}$ |

> Similar to measurements in single-qubit systems, the assumption of normalization of the original wave function is required in order to ensure that the sum of all the outcome probabilities is 1.

### Analyze 1. Multi-qubit measurement in computational basis

You are given a two-qubit system in the following state:
$$\ket \psi =  \tfrac{2}{3}\ket {00} + \tfrac{1}{3} \ket {01} + \tfrac{2}{3}\ket {11}$$

If all the qubits are measured simultaneously in the computational basis, what are the outcome probabilities?

<details>
<summary><b>Solution</b></summary>

The wave function $\ket{\psi}$ is normalized, since $\left(\frac{2}{3}\right)^2 + \left(\frac{1}{3}\right)^2 + \left(\frac{2}{3}\right)^2 = 1$. Hence, the probabilities of measuring each of the computational basis states is simply the square of the absolute value of the corresponding coefficients. That is, the probabilities of measuring $00$, $01$ and $11$ are $\frac{4}{9}$, $\frac{1}{9}$ and $\frac{4}{9}$, respectively, and the probability of measuring the basis state $10$ that is not part of the superposition is $0$:

| Measurement outcome | Probability of outcome |
| - | - |
| $00$ | $\left( \frac{2}{3}\right)^2 = \frac{4}{9}$ |
| $01$ | $\left( \frac{1}{3}\right)^2 = \frac{1}{9}$ |
| $10$ | $\left( 0\right)^2 = 0$ |
| $11$ | $\left( \frac{2}{3}\right)^2 = \frac{4}{9}$ |

</details>

### Analyze 2. Multi-qubit measurement in Pauli X basis

You are given a two-qubit system in the same state as in the previous exercise:
$$\ket \psi =  \tfrac{2}{3}\ket {00} + \tfrac{1}{3} \ket {01} + \tfrac{2}{3}\ket {11}$$

If all the qubits are measured simultaneously in the Pauli X basis, that is, in the $\{ \ket{++}, \ket{+-}, \ket{-+}, \ket{--}\}$ basis, what are the outcome probabilities?

<details>
<summary><b>Solution</b></summary>

Using the expressions $\ket{0} = \frac{1}{\sqrt{2}} \big( \ket{+} + \ket{-} \big)$ and $\ket{1} = \frac{1}{\sqrt{2}} \big( \ket{+} - \ket{-} \big)$, you first express $\ket{\psi}$ in the Pauli X basis. This gives you
$$\ket \psi =  \frac{2}{3}\ket {00} + \frac{1}{3} \ket {01} + \frac{2}{3}\ket {11} =$$

$$= \frac{2}{3} \big[ \frac{1}{\sqrt{2}}\big(\ket{+} + \ket{-}\big) \otimes \frac{1}{\sqrt{2}} \big(\ket{+} + \ket{-}\big) \big] + \frac{1}{3} \big[ \frac{1}{\sqrt{2}}\big(\ket{+} + \ket{-}\big) \otimes \frac{1}{\sqrt{2}} \big(\ket{+} - \ket{-}\big) \big] + \frac{2}{3} \big[ \frac{1}{\sqrt{2}}\big(\ket{+} - \ket{-}\big) \otimes \frac{1}{\sqrt{2}} \big(\ket{+} - \ket{-}\big) \big] =$$

$$= \frac{1}{3} \big[ \big(\ket{+} + \ket{-}\big) \otimes \big(\ket{+} + \ket{-}\big) \big] + \frac{1}{6} \big[ \big(\ket{+} + \ket{-}\big) \otimes \big(\ket{+} - \ket{-}\big) \big] + \frac{1}{3} \big[ \big(\ket{+} - \ket{-}\big) \otimes \big(\ket{+} - \ket{-}\big) \big] =$$

$$= \frac{1}{3} \big[ \ket{++} + \ket{+-} + \ket{-+} + \ket{--} \big] + \frac{1}{6} \big[ \ket{++} - \ket{+-} + \ket{-+} - \ket{--} \big] + \frac{1}{3} \big[ \ket{++} - \ket{+-} - \ket{-+} + \ket{--} \big] =$$

$$= (\frac{1}{3} + \frac{1}{6} + \frac{1}{3})\ket{++} + (\frac{1}{3} - \frac{1}{6} - \frac{1}{3})\ket{+-} + (\frac{1}{3} + \frac{1}{6} - \frac{1}{3})\ket{-+} + (\frac{1}{3} - \frac{1}{6} + \frac{1}{3})\ket{--} =$$

$$= \frac{5}{6}\ket{++} - \frac{1}{6}\ket{+-} + \frac{1}{6}\ket{-+} + \frac{1}{2}\ket{--}$$

After this, the probabilities of measuring each of the four basis vectors is given by the square of the absolute value of its amplitude in the superposition:

| Measurement outcome | Probability of outcome |
| - | - |
| $++$ | $\left( \frac{5}{6}\right)^2 = \frac{25}{36}$ |
| $+-$ | $\left( -\frac{1}{6}\right)^2 = \frac{1}{36}$ |
| $-+$ | $\left( \frac{1}{6}\right)^2 = \frac{1}{36}$ |
| $--$ | $\left( \frac{1}{2}\right)^2 = \frac{1}{4}$ |

</details>

## Measuring each qubit in a system sequentially

As described in the previous sections, in theory it's possible to measure all the qubits in an $n$-qubit system simultaneously in an orthogonal basis. The post-measurement state of the qubits is then exactly one of the $2^n$ possible basis states.

In practice, this is implemented by measuring all the qubits one after another. For example, if one wants to measure a two-qubit system in the computational basis, one can implement this by first measuring the first qubit in the computational basis to obtain $0$ or $1$, and then measuring the second qubit in the computational basis. This can result in one of the four possible outcomes: $00, 01, 10, 11$.

This can be generalized to measurements in other bases, such as the 2-qubit Pauli X basis $\ket{++}, \ket{+-}, \ket{-+}, \ket{--}$, and the bases for larger numbers of qubits.

> Note that measurement of all qubits sequentially can only be done in orthogonal bases $\{ \ket{b_i}\}$, such that each $\ket{b_i}$ is a **tensor product state**. That is, each $\ket{b_i}$ must be of the form $\ket{v_0} \otimes \ket{v_1} \dotsc \otimes \ket{v_{n-1}}$, with each $\ket{v_j}$ being a single-qubit basis state.
For example, for the two-qubit Pauli X basis $\ket{++}, \ket{+-}, \ket{-+}, \ket{--}$ each basis state is a tensor product of states $\ket{+}$ and $\ket{-}$, which form a single-qubit basis state.
>
> Measuring in orthogonal bases which contain states which aren't tensor product states, such as the Bell basis, are trickier to implement, and require appropriate unitary rotations in addition to measuring all qubits one after another.
> Such measurements aren't discussed in this kata.
>
> If measurements are restricted to tensor product states, the distinction between measuring all the qubits simultaneously versus one after another is not important for an ideal quantum computer: in terms of the outcomes and measurement probabilities, both approaches are identical. Furthermore, as long as all the qubits are measured, the sequence in which they are measured is also inconsequential. These factors can be important in the case of real quantum computers with imperfect qubits, but the discussion in this kata is limited to ideal systems.

## Measurement statistics for qubit-by-qubit full measurement

The following example demonstrates how to perform multiple measurements on a qubit register in Qiskit. It prepares a qubit in the state from the "Analyze 1" exercise, $\ket \psi =  \frac{1}{3}\ket {00} + \frac{2}{3} \ket {01} + \frac{2}{3}\ket {11}$, and then measures both qubits in the computational basis.

In Qiskit, the `measure` method of the `QuantumCircuit` object can be used to measure multiple qubits, one by one, and to write measurement results into the corresponding slots of a `ClassicalRegister` register.

After you run the circuit on a simulator, you can extract measurement outcomes using the `get_counts` method, same as for the single-qubit state.

> Notice that Qiskit reverses the bit string of outcomes, so that they use big-endian notation while all the internal computations use little-endian notation. You can see this in the demo: the basis state $\ket{10} = \ket{1}_{dec}$ (little-endian) had amplitude of $0$ in the call to `initialize`, but in the measurement outcomes the key that corresponds to it is `01` (big-endian). You have to keep that in mind when measuring multiple qubits and analyzing the results.

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

# Create the simulator instance
simulator = AerSimulator(method='statevector')

# Define the quantum circuit
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
circ = QuantumCircuit(qr, cr)
# Initialize the qubits in the state 1/3|00⟩ + 2/3|01⟩ + 2/3|11⟩
circ.initialize([1/3, 0, 2/3, 2/3], qr)

# Measure the qubits
circ.measure(qr, cr)

# Run the simulation and get the measurement statistics
n_shots = 100
res = simulator.run(circ, shots=n_shots).result()
counts = res.get_counts()

expected_probabilities = [1/9, 0, 4/9, 0, 4/9]

print("| Measurement outcome | Theoretical probability | Simulated probability |")
print("| - | - |")
for outcome in range(4):
    key = str(outcome // 2) + str(outcome % 2)
    print(f"| {key} | {expected_probabilities[outcome]} | {counts.get(key, 0)} |")


| Measurement outcome | Theoretical probability | Simulated probability |
| - | - |
| 00 | 0.1111111111111111 | 4 |
| 01 | 0 | 0 |
| 10 | 0.4444444444444444 | 47 |
| 11 | 0 | 49 |


Full measurements can also be used to identify the state of the system, if it is guaranteed to be in one of several possible orthogonal states, same as for the single-qubit case.

## Exercise 1. Distinguish four basis states

**Inputs:** 

1. A quantum circuit.
2. Two qubits which are guaranteed to be in one of the four basis states ($\ket{00}$, $\ket{01}$, $\ket{10}$, or $\ket{11}$), represented as `QuantumRegister` of length 2.
3. A classical register of length 2.

**Goal:** Figure out which state the qubits are in:

* 0 if the qubits are in the $\ket{00}$ state,
* 1 if they are in the $\ket{10}$ state,
* 2 if they are in the $\ket{01}$ state,
* 3 if they are in the $\ket{11}$ state.

In [ ]:
@exercise
def measure_basis_state(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    # Write your code here
    ...

## Partial measurements

For a system with $n>1$ qubits, it's possible to measure $m<n$ qubits one after another. The number of measurement outcomes is then $2^m$ instead of $2^n$. The probabilities of each of the outcomes and the post-measurement states of the qubits can be found using the projection formalism for measurements.

First, recall the concept of projection operators introduced in the "Single-Qubit Measurements" kata. Measurements are modeled by orthogonal projection operators - matrices that satisfy
$$P^2 = P^\dagger = P$$
Consider an $n$-qubit system in a state $\ket{\psi}$, for which the first $m<n$ qubits are measured in an orthogonal basis $\{ \ket{b_0} , \ket{b_1}, \dotsc, \ket{b_{2^m-1}}\}$ corresponding to the $m$ qubits being measured. Then, you define $2^m$ projectors corresponding to each of the $\ket{b_i}$ states as

$$P_i = \ket{b_i} \bra{ b_i} \otimes \mathbb{1}_{n-m} $$

where $\mathbb{1}_{n-m}$ is the identity operator over the remaining $(n-m)$ qubits.

> The symbol $\otimes$ represents the tensor product or the Kronecker product of two matrices. It's different from the usual matrix multiplication.
> In the current context, $\ket{b_i} \bra{ b_i} \otimes \mathbb{1}_{n-m}$ simply means that
>
> - The operator $\ket{b_i} \bra{ b_i}$ acts only on the $m$ qubits being measured.
> - The effect of $P_i$ on the remaining qubits is $\mathbb{1}_{n-m} $, that is, the identity operator.

Analogous to the case for measurements for single-qubit systems, the rules for partial measurement probabilities and outcomes can be summarized as follows:

- When a measurement is done, one of these projectors is chosen randomly. The probability of choosing projector $P_i$ is $\big|P_i\ket{\psi}\big|^2$.
- If the projector $P_i$ is chosen, the measurement outcome is $b_i$, and the state of the system after the measurement is given by

$$
\frac{P_i \ket{\psi}}{\big|P_i \ket{\psi}\big|}.
$$

For example, consider a two-qubit system in the state $\ket \psi = \frac{1}{\sqrt{2}}\ket{01} - \frac{1}{\sqrt 2}\ket{10}$. Consider a measurement of the first qubit in the computational basis, that is, in the $\{\ket 0 , \ket 1 \}$ basis. Then, you have two projectors that represent this measurement:
$$P_0 = \ket 0\bra 0 \otimes \mathbb{1}$$
$$P_1 = \ket 1 \bra 1 \otimes \mathbb{1}$$

The action of $P_0$ on $\ket \psi$ is

$$P_0 \ket \psi = \left(\ket 0\bra 0 \otimes \mathbb{1}\right) \frac{1}{\sqrt 2}\big(\ket{01} - \ket{10}\big) =$$

$$= \tfrac{1}{\sqrt 2} \big( \ket 0 \braket{0|0} \otimes \mathbb{1} \ket{1} - \ket 0 \braket{0|1} \otimes \mathbb{1} \ket 0 \big) = \tfrac{1}{\sqrt 2} \ket{01}$$

Similarly, you obtain
$$P_1 \ket\psi = -\frac{1}{\sqrt 2} \ket{10}$$

Clearly, you have $\big|P_0 \ket \psi\big| = \big|P_1 \ket \psi\big| = \frac{1}{2}$ in this case. Thus, the probabilities of measuring $0$ and $1$ are both $0.5$, with the post-measurement states of system being $\ket{01}$ and $\ket{10}$, respectively.

> Similar to the case of single-qubit system measurements, the applicability of the formalism above requires the state of the multi-qubit system, $\ket \psi$, to be normalized. This is required to ensure that all the probabilities of individual outcomes add up to 1.

### Analyze 3: Partial measurement probabilities for the Hardy state

Consider a two-qubit system in the Hardy state $\ket \psi = \tfrac{1}{\sqrt{12}} \big(3\ket{00} + \ket{01} + \ket{10} + \ket{11}\big)$.

If only the first qubit is measured in the computational basis, what are the probabilities of the outcomes, and the post-measurement states of the system?

<details>
<summary><b>Solution</b></summary>

A measurement outcome of $0$ on the first qubit corresponds to the projection operator $P_0 = \ket{0}\bra{ 0} \otimes \mathbb{1}$. Applying it to the state $\ket \psi$ gives you
$$\big|P_0 \ket{\psi}\big|^2 = \big|\tfrac{1}{\sqrt{12}} \left(3\ket {00} + \ket{01}\right) \big|^2 = \tfrac{5}{6}$$
and
$$\tfrac{P_0 \ket{\psi}}{\big|P_0 \ket{\psi}\big|} = \tfrac{1}{\sqrt{10}} \left( 3\ket{00} + \ket{01}\right)$$

Similarly, $P_1 = \ket{1} \bra{ 1 } \otimes \mathbb{1}$ is the projector corresponding to a measurement outcome of $1$ on the first qubit. Applying $P_1$ on $\ket{\psi}$ gives us $\big|P_1 \ket{\psi}\big|^2 = \tfrac{1}{6}$ and

$$\tfrac{P_1 \ket{\psi}}{\big|P_1 \ket{\psi}\big|} = \tfrac{1}{\sqrt{2}} \left(\ket{10} + \ket{11}\right)$$

| Measurement outcome | Probability of outcome | Post-measurement state |
| - | - | - |
| $0$ | $\tfrac{5}{6}$ | $\tfrac{1}{\sqrt{10}} \left( 3\ket{00} + \ket{01}\right)$ |
| $1$ | $\tfrac{1}{6}$ | $\tfrac{1}{\sqrt{2}} \left(\ket{10} + \ket{11}\right)$ |

</details>

## Measurement statistics for partial measurement

This demo shows that the simulated outcome probabilities and post-measurement outcomes match the theoretical values obtained using the projection operators as described above. It uses the Hardy state $\ket \psi = \tfrac{1}{\sqrt{12}} \big(3\ket{00} + \ket{01} + \ket{10} + \ket{11}\big)$ from the "Analyze 3" exercise and do a computational basis measurement on the first qubit.

The simulated and theoretical measurement probabilities aren't expected to match exactly, but should be close to each other, since measurement is probabilistic. However, the post-measurement states from the simulation should match the expected states calculated in the exercise precisely, since partial state collapse isn't a probabilistic process.

In [9]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

# Create the simulator instance
simulator = AerSimulator(method='statevector')

# Define the quantum circuit
qr = QuantumRegister(2)
cr = ClassicalRegister(1)
circ = QuantumCircuit(qr, cr)
# Initialize the qubits in the Hardy state
circ.initialize([3, 1, 1, 1], qr, normalize=True)

# Measure the first qubit
circ.measure(qr[0], cr)

# Run the simulation and get the measurement statistics
n_shots = 100
res = simulator.run(circ, shots=n_shots).result()
counts = res.get_counts()

expected_probabilities = [0.833, 0.167]

print("| Measurement outcome | Theoretical probability | Simulated probability |")
print("| - | - |")
for outcome in range(2):
    key = str(outcome)
    print(f"| {key} | {expected_probabilities[outcome]} | {counts.get(key, 0)} |")


| Measurement outcome | Theoretical probability | Simulated probability |
| - | - |
| 0 | 0.833 | 83 |
| 1 | 0.167 | 17 |


In certain situations, it's possible to distinguish between orthogonal states of multi-qubit systems using partial measurements, as illustrated in the next exercise.

## Exercise 2. Distinguish orthogonal states using partial measurements

**Inputs:** 

1. A quantum circuit.
2. Two qubits which are guaranteed to be in one of the two orthogonal states, represented as `QuantumRegister` of length 2.
3. A classical register of length 2.

**Goal:** Figure out which state the qubits are in:

* 0 if the qubits are in the $\ket{+-}$ state,
* 1 if they are in the $\ket{--}$ state.

In [ ]:
@exercise
def measure_plusminus_state(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    # Write your code here
    ...

## Measurements and entanglement

Entanglement has an effect on the measurement statistics of the system. If two qubits are entangled, then their measurement outcomes will be correlated, while separable states (which are by definition not entangled) have uncorrelated measurement outcomes.

> It's useful to revisit the concepts of entanglement and separable states, which were introduced in the "Multi-Qubit Systems" kata. Consider a system of $n>1$ qubits, which is divided into two parts: A, consisting of $m$ qubits, and B, consisting of the remaining $n-m$ qubits. The state $\ket \psi$ of the entire system is separable if it can be expressed as a tensor product of the states of parts A and B:
> $$\ket \psi = \ket {\phi_A} \otimes \ket{\phi_B}$$
> Here $\ket{\phi_A}$ and $\ket{\phi_B}$ are wave functions that describe parts $A$ and $B$, respectively. If it isn't possible to express $\ket \psi$ in such a form, then system A is entangled with system B.

Consider a measurement on the subsystem $A$ of a separable state. Let the measurement be done in a basis $\{ \ket{b_0},\dotsc,\ket{b_{2^m-1}}\}$. According to the projection formalism, a projection operator $P_i = \ket{b_i}\bra{b_i} \otimes \mathbb{1}$ is chosen randomly. The corresponding post-measurement state of the system is then given by

$$\ket{\psi}_{i} \equiv \frac{P_i \ket{\psi}}{\big|P_i \ket{\psi}\big|}
= \frac{ \ket{b_i} \braket{b_i|\phi_A} \otimes \ket{\phi_B} }{\big| \ket{b_i}\braket{b_i|\phi_A} \otimes \ket {\phi_B} \big|}=$$

$$= \frac{\braket{b_i|\phi_A} \cdot \ket{b_i} \otimes \ket {\phi_B}}{\big|\ket{b_i}\big| \cdot \braket{b_i|\phi_A} \cdot \big| \ket {\phi_B}\big|}
= \ket{b_i} \otimes \ket{\phi_B}$$

Thus, the state of subsystem $B$ after the measurement is $\ket{\phi_B}$ independently of the outcome $i$ of the measurement on the first qubit. The results of a subsequent measurement on subsystem $B$, including outcome probabilities, will be independent of the result of the first measurement. In other words, the outcomes of the two measurements will be uncorrelated.

On the other hand, if the system is entangled, then the measurement outcomes will be correlated, in a manner dictated by the bases chosen for the measurements on the two subsystems. The following exercise illustrates this phenomenon.

### Analyze 4: Sequential measurements on an entangled state and a separable state

Consider two two-qubit states:

- The Bell state $\ket{\Phi^{+}} = \frac{1}{\sqrt{2}} \big (\ket{00} + \ket{11}\big)$.
- A state $\ket \Theta = \frac{1}{2} \big( \ket{00} + \ket{01} + \ket{10} + \ket{11} \big)$.

For both states, consider a measurement on the first qubit, followed by a measurement on the second qubit, both done in the computational basis. For which state can you expect the measurement outcomes to be correlated? Verify by calculating the sequential measurement probabilities explicitly for both states.

<details>
<summary><b>Solution</b></summary>

<b>The Bell state</b>: If the measurement outcome on the first qubit is $0$, a subsequent measurement on the second qubit <i>always</i> results in an outcome of $0$, with probability $1$. Similarly, if the measurement outcome on the first qubit is $1$, then the second qubit measurement always results in $1$. Thus, sequential measurements are perfectly <b>correlated</b>.

<b>Separable state $\ket \Theta$</b>: Irrespective of whether the first qubit measurement outcome is $0$ of $1$ (each of which occurs with a probability of $0.5$), a subsequent measurement on the second qubit results in an outcome of $0$ or $1$ (both with a probability of $0.5$). Thus, sequential measurements are perfectly <b>uncorrelated</b>.

This aligns with the fact that the Bell state is entangled, while the state $\ket{\Theta}$ is separable and can be expressed as $\ket \Theta = \ket + \otimes \ket +$.
</details>


# Conclusion

Congratulations! In this kata you learned how to apply measurements on multi-qubit systems. Here are a few key concepts to keep in mind:

- Full measurements: you measure all the qubits simultaneously in an orthogonal basis ($2^n$ possible outcomes).
- Partial measurements: you measure $m$ qubits out of $n$, for $m < n$ ($2^m$ possible outcomes).